In [ ]:
# imports
import pysam
from pathlib import Path

In [ ]:
def get_bam_stats(bamfile):
    """Read alignment stats from BAM file"""
    bam_stats = {}
    bam_stats["all_primary_alignments"] = int(pysam.view("-c", "-F", "256", bamfile))
    bam_stats["all_mapped_pairs"] = int(pysam.view("-c", "-F", "256", "-F", "8", bamfile))
    bam_stats["proper_pairs"] = int(pysam.view("-c", "-f", "2", bamfile))
    bam_stats["proper_pairs_unique"] = int(pysam.view("-c", "-f", "2", "-q", "1", bamfile))
    bam_stats["proper_pairs_unique_sense"] = int(pysam.view("-c", "-f", "2", "-q", "1", "-f", "64", "-F", "16", bamfile)) * 2
    bam_stats["proper_pairs_unique_antisense"] = int(pysam.view("-c", "-f", "2", "-q", "1", "-f", "64", "-f", "16", bamfile)) * 2
    return(bam_stats)

In [ ]:
stats = {}
with open(snakemake.output[0], "w") as output:
    output.write("sample\talignment category\treads (individual alignments)\tfragments (read pairs)\n")
    for filename in (snakemake.input.bam):
        stats[filename] = get_bam_stats(filename)
        for stat, read_count in stats[filename].items():
            if stat == "all_primary_alignments":
                fragment_count = None
            else:
                fragment_count = int(read_count / 2)
            output_line = f"{Path(filename).stem}\t{stat}\t{read_count}\t{fragment_count}"
            output.write(f"{output_line}\n")
            print(output_line)